# TODO: convert this to .py and write shell script

In [1]:
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

# import pandas as pd
# pd.set_option('display.max_columns', 500)


import os
from os.path import expanduser

import math
from math import sin

# plt.style.use('dark_background')

from scipy.io import loadmat
import scipy
from scipy import signal
from scipy.fftpack import fft, ifft
from scipy.signal import hilbert, chirp
from scipy.signal import find_peaks
from scipy.interpolate import interp1d

import argparse
import json

import pywt
from pywt import scale2frequency

import emd

i_seed = 0

    
import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../PatchWand/') # add this line so Data and data are visible in this file

from plotting_tools import *
from preprocessing import *
# from data_pulling import *
from setting import *
from surrogate_extraction import *
from dataIO import *
from filters import *
from spectral_module import *
import datetime
import time


# checklist 1: uncomment matplotlib.use('Agg')
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot


# checklist 2: comment out all magic command
from importlib import reload
%load_ext autoreload
%autoreload 2

# please design this HP carefully, use inspect_TF_rep to find optimal value
# length_medfilt = 55


In [2]:
parser = argparse.ArgumentParser(description='RR_estimate')
parser.add_argument('--input_folder', metavar='input_folder', help='input_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')
parser.add_argument('--windowing_params', metavar='windowing_params', help='windowing_params',
                    default='windowing_params.json')

parser.add_argument('--modalities', metavar='modalities', help='modalities',
                    default='ECG_AM')

parser.add_argument('--script_id', metavar='script_id', help='script_id',
                    default='?')


# checklist 3: comment first line, uncomment second line
args = parser.parse_args(['--input_folder', '../../data/stage3/resp/', 
                          '--output_folder', '../../data/stage3-1_windowing/CDC_dataset/',
                          '--windowing_params', 'windowing_params.json'])

# args = parser.parse_args(['--input_folder', '/home/mchan/Estimation_RR/covid/results/stage2/', 
#                           '--output_folder', '../../data/stage3-1_windowing/GT_dataset/',
#                           '--windowing_params', 'windowing_params.json'])


# args = parser.parse_args()
print(args)

Namespace(input_folder='../../data/stage3/resp/', modalities='ECG_AM', output_folder='../../data/stage3-1_windowing/CDC_dataset/', script_id='?', windowing_params='windowing_params.json')


In [3]:
inputdir = args.input_folder
outputdir = args.output_folder
windowing_params = args.windowing_params

if not os.path.exists(outputdir):
    os.makedirs(outputdir)
print('will export data to', outputdir)



will export data to ../../data/stage3-1_windowing/CDC_dataset/


In [4]:
with open(windowing_params) as json_file:
    windowing_params_list = json.load(json_file)

windowing_params = windowing_params_list[0]

# get items from windowing_params dict

In [5]:
if windowing_params.get('remove_outliers'): 
    clean_str = '_cleaned'
else:
    clean_str = ''

normalization = windowing_params['normalization']

if normalization:
    norm_str = 'norm'
else:
    norm_str = ''
    
if windowing_params.get('debug_TF'): 
    debug_TF = windowing_params['debug_TF']
else:
    debug_TF = False



In [6]:
outputdir = outputdir+'win{}_overlap{}_seq{}_{}{}/'.format(str(windowing_params['window_size']), \
                                                                                int(windowing_params['overlap']*100), \
                                                                                str(windowing_params['seq_len']), \
                                                                                    clean_str,
                                                                                    norm_str,
                                                                                    )

if not os.path.exists(outputdir):
    os.makedirs(outputdir)
print('will export data to', outputdir)


if not os.path.exists(outputdir+'sur_resp/'):
    os.makedirs(outputdir+'sur_resp/')

will export data to ../../data/stage3-1_windowing/CDC_dataset/win60_overlap95_seq20_norm/


In [7]:
if 'GT_dataset' in outputdir:

    home = expanduser("~")

    filename_subinfo = home+'/'+ 'Estimation_RR/covid/Mobashir_CosmedWearable/Cosmed Wearable Subject Information.xlsx'
    df_subinfo = pd.read_excel(filename_subinfo) 
    df_subinfo

    df_subinfo = df_subinfo.rename(columns={'Height in cm': 'Height', 'Weight in Kg': 'Weight', 'Subject ID':'Subject_ID'})

    df_subinfo['Subject_ID'] = df_subinfo['Subject_ID'].apply(lambda x: int(x.split('M')[0].split('F')[0].split('CW')[1]))
    df_subinfo = df_subinfo[['Subject_ID', 'Age', 'Height', 'Weight', 'Gender']].drop_duplicates().reset_index(drop=True)
    df_subinfo['BMI'] = df_subinfo['Weight'].values / (df_subinfo['Height'].values/100)**2
    df_subinfo['Fitzpatrick'] = -1
    
elif 'CDC_dataset' in outputdir:

    filename_subinfo = '../../data/raw/FS Subject Information.xlsx'
    df_subinfo = pd.read_excel(filename_subinfo, sheet_name='Subject info') 

    i_sub_last = np.where(df_subinfo['Subject_ID'].isnull()==True)[0][0]
    df_subinfo = df_subinfo[:i_sub_last]

    df_subinfo = df_subinfo[['Subject_ID', 'Age', 'Height(cm)', 'Weight(Kg)', 'Gender(0=Male,1=Female)', 'Fitzpatrick', '6MWT distance (m)', '6MWT-R distance (m)', 'Fitzpatrick']]
    df_subinfo = df_subinfo.rename(columns={'Height(cm)': 'Height', 'Weight(Kg)': 'Weight', 'Gender(0=Male,1=Female)': 'Gender'})
    df_subinfo['BMI'] = df_subinfo['Weight'].values / (df_subinfo['Height'].values/100)**2
    df_subinfo['Subject_ID'] = df_subinfo['Subject_ID'].astype(int)
    

# get demographic data

# retrieve signal and labels

# retrieve the data
    TODO: include ECG PCA
    TODO: go back to stage3 and check SCG AMpt again (value don't seem right)
    

In [8]:
outputdir

'../../data/stage3-1_windowing/CDC_dataset/win60_overlap95_seq20_norm/'

# retrieve signal and labels from CDC or the GT dataset

In [9]:
downsample_factor

10

In [10]:
show_RR_est = True

if 'GT_dataset' in outputdir:
    print('woring on GT_datatset')
    windowing_params['Fs'] = FS_RESAMPLE_resp
    Fs = FS_RESAMPLE_resp



    df_resp_subs = pd.DataFrame()

    for sub_file in os.listdir(inputdir):
        if '.feather' in sub_file and 'error' not in sub_file and 'predictions' not in sub_file:
            df = pd.read_feather(inputdir+sub_file) 
            subject_id = sub_file.split('_')[1].split('.')[0]
            df = df.drop(columns=['ECG', 'accelX', 'accelY', 'accelZ', 'br_binary', 'rer', 'vevco2_slope', 'i_R_peaks', 'i_S_peaks'])
            df = df.rename(columns={"br": "RR_cosmed", "vt": "VT_cosmed", "heart_rate_cosmed": "HR_cosmed"})
#             sys.exit()

            
            Fs_df = np.median((1/np.diff(df['time'].values)).astype(int))
            downsample_factor = int(Fs_df//FS_RESAMPLE_resp)
#             downsample_factor = 1
    
            Fs = Fs_df/downsample_factor
            windowing_params['Fs'] = Fs
            
            df = df[::downsample_factor]

            print('working on {}...'.format(subject_id))

            # add the task column
            df['task'] = 'Recovery Treadmill'
            df['task_id'] = 101


            df_resp_subs = df_resp_subs.append(df)


            print(df.shape[0]/Fs,'sec')
            
    df_resp_subs['domain'] = 'GT_datatset'
    df_resp_subs['subject_id'] = df_resp_subs['subject_id'].astype(int)



elif 'CDC_dataset' in outputdir:
    print('woring on CDC_datatset')
    df_resp_subs = pd.read_feather(inputdir+'df_resp_subs.feather')
    df_resp_subs['subject_id'] = df_resp_subs['subject_id'].astype(int)
    df_resp_subs['domain'] = 'CDC_datatset'
    
#     df_RR_est = pd.read_feather(inputdir+'sur_resp/df_RR_est.feather') 
#     df_resp_PCC = pd.read_feather(inputdir+'sur_resp/df_resp_PCC.feather')

#     stage3_dict = data_loader('stage3_dict', inputdir).item()
    Fs = FS_RESAMPLE_resp

    windowing_params['Fs'] = Fs
    
    
df_RR_est = get_df_RR_est(df_resp_subs).reset_index(drop=True)
df_resp_PCC = get_df_resp_PCC(df_resp_subs).reset_index(drop=True)
    

df_RR_est.to_feather(outputdir+'sur_resp/df_RR_est.feather')
df_resp_PCC.to_feather(outputdir+'sur_resp/df_resp_PCC.feather')


if show_RR_est:
    for task_name in df_RR_est['task'].unique():
        plot_RR_est_error(df_RR_est[df_RR_est['task']==task_name],fig_name='RR_est_error_'+task_name, outputdir=outputdir+'sur_resp/', show_plot=False)

    for task_name in df_RR_est['task'].unique():
        plot_HM(df_RR_est[df_RR_est['task']==task_name], 'surrogate_names', 'subject_id', 'RR_mae', fig_name='RR_mae_'+task_name, outputdir=outputdir+'sur_resp/', show_plot=False)

print('done!')

woring on CDC_datatset
done!


In [11]:
df_resp_subs.keys()

Index(['time', 'ECG_SR', 'ECG_AMr', 'ECG_AMs', 'ECG_AMpt', 'ECG_FM', 'SCG_AM',
       'SCGxyz_AMpt', 'SCG_AMpt', 'SCGx_AMpt', 'SCGy_AMpt', 'accelX_resp',
       'accelY_resp', 'accelZ_resp', 'ppg_g_1_cardiac_max',
       'ppg_g_1_cardiac_min', 'ppg_g_2_cardiac_max', 'ppg_g_2_cardiac_min',
       'ppg_r_1_cardiac_max', 'ppg_r_1_cardiac_min', 'ppg_r_2_cardiac_max',
       'ppg_r_2_cardiac_min', 'ppg_ir_1_cardiac_max', 'ppg_ir_1_cardiac_min',
       'ppg_ir_2_cardiac_max', 'ppg_ir_2_cardiac_min', 'ppg_g_1_resp',
       'ppg_g_2_resp', 'ppg_r_1_resp', 'ppg_r_2_resp', 'ppg_ir_1_resp',
       'ppg_ir_2_resp', 'RR_cosmed', 'resp_cosmed', 'subject_id', 'task',
       'HR_cosmed', 'VT_cosmed', 'br_sim_sig', 'domain'],
      dtype='object')

# comptue ST windows

In [12]:
# surrogate_name = surrogate_names[0]
# surrogate_name

In [13]:
def extract_windows_signals(ts_Pxx, t_sig, sig, Fs, windowing_params=None):

    start_offset = -int(windowing_params['window_size']/2*Fs)
    end_offset = int(windowing_params['window_size']/2*Fs)

    # identify the center of the Pxx in terms of indices
    # t_sig may not start at 0, so need to substract it from t_sig[0] to get the right indices
    i_Pxx = ( (ts_Pxx-t_sig[0]) *Fs).astype(int)

    # segment the signal
    sig_segments, _ = beat_segmentation(sig, i_Pxx, start_offset=start_offset, end_offset=end_offset)
    sig_segments = sig_segments.T # dim = # of segments, # of samples in each segment
#     print(sig_segments.shape)

    # arrange the signal segments into sequence
    N_windows_seq = windowing_params['seq_len']

    N_windows = sig_segments.shape[0] - (N_windows_seq - 1)
    i_starts = np.arange(N_windows)
    ts_sig_segments_seq, sig_segments_seq = Pxx_segmentation(ts_Pxx, sig_segments, i_starts, start_offset=0, end_offset=N_windows_seq)

#     ts_sig_segments_seq.shape, sig_segments_seq.shape

    return ts_sig_segments_seq, sig_segments_seq




In [15]:
# df_resp_sub['task'].unique()

In [16]:
# for task in df_resp_sub['task'].unique():
#     if 'Recovery' not in task:
#         if 'Stair' in task:
#             continue
#     print(task)

In [17]:
# windowing_param

In [18]:
# length_medfilt = 2001
norm_param = {
    'mode': 'EMD',
}
norm_param = {
    'mode': 'medfilt',
#     'length_medfilt': 2001, 
    'length_medfilt': math.ceil(Fs * 4 / 2.) * 2 + 1,
#     'length_medfilt': 21, 
}

windowing_params['norm_param'] = norm_param

# show_surrogate_names = ['ECG_SR', 'ECG_AMr', 'ECG_AMs', 'ECG_AMpt', 'ECG_FM', 'SCG_AM',
#                         'SCGxyz_AMpt', 'SCG_AMpt', 'SCGx_AMpt', 'SCGy_AMpt',
#                         'ppg_g_1_resp', 'ppg_g_2_resp', 'ppg_r_1_resp', 'ppg_r_2_resp',
#                         'ppg_ir_1_resp', 'ppg_ir_2_resp']

show_surrogate_names = ['ECG_SR',
                        'SCGxyz_AMpt', 'SCG_AMpt', 'SCGx_AMpt', 'SCGy_AMpt',
                        'accelX_resp', 'accelY_resp', 'accelZ_resp',
                        'ppg_g_1_resp', 'ppg_g_2_resp', 'ppg_ir_1_resp', 'ppg_ir_2_resp']

surrogate_names = show_surrogate_names

inspect_TF = True


label_all = []
data_all = []
raw_all = []
meta_all = []
ts_all = []
hr_all = []
Pxx_names = surrogate_names

for subject_id in df_resp_subs['subject_id'].unique():
    
#     if subject_id != 1:
#         continue
    
    print('working on subject', subject_id)
    df_resp_sub = df_resp_subs[df_resp_subs['subject_id']==subject_id]

    for task_name in df_resp_sub['task'].unique():
        
#         if task_name=='Stair':
        if 'Recovery' not in task_name:
            if 'Stair' in task_name:
                continue
        
        df_resp = df_resp_sub[df_resp_sub['task']==task_name]

        t_sig = df_resp['time'].values
        
#         sys.exit()
        
        resp_sig =  df_resp['br_sim_sig'].values

#         surrogate_names = np.asarray(df_resp.columns)
#         surrogate_names = surrogate_names[(surrogate_names!='time') & (surrogate_names!='RR_cosmed') & (surrogate_names!='subject_id') & (surrogate_names!='task')]
        column_names = df_resp.columns
#         surrogate_names = get_surrogate_names(column_names)
        surrogate_names = show_surrogate_names
#         sys.exit()
        
        
        N_sur = len(surrogate_names)

        Pxx_list = []
        raw_list = []

        for surrogate_name in surrogate_names:
            
#             if surrogate_name!='ppg_g_2_resp':
#                 continue
                


#             print(task_name, surrogate_name)
            
#             if surrogate_name!='SCG_AMpt':
#                 continue
                
            sig = df_resp[surrogate_name].values
        
        
        
#             sys.exit()

        
        #     ts_Pxx_seq, Pxx_seq_surrogate, freq = extract_windows(df_resp['time'].values, df_resp[surrogate_name].values, Fs, norm_param)
            ts_Pxx_seq, Pxx_seq_surrogate, freq, ts_Pxx, Pxx = extract_windows(t_sig, sig, Fs, windowing_params)
            if inspect_TF:
                if surrogate_name in show_surrogate_names:
                    inspect_TF_rep(df_resp['time'].values, df_resp[surrogate_name].values, resp_sig, windowing_params, task_name=task_name, subject_id=subject_id, sig_name=surrogate_name, outputdir=outputdir+surrogate_name+'/', show_plot=False)
#                 sys.exit()
            Pxx_list.append(Pxx_seq_surrogate)


            ts_sig_segments_seq, sig_segments_seq = extract_windows_signals(ts_Pxx, t_sig, sig, Fs, windowing_params=windowing_params)
            raw_list.append(sig_segments_seq)
            

        Pxx_matrix = np.stack(Pxx_list, axis=3).transpose(0,3,1,2) # dimension (# of sample >100, # of surrogates = 13, # of time steps = 20, # of spectral features = 46)
        raw_matrix = np.stack(raw_list, axis=3).transpose(0,3,1,2) # dimension (# of sample >100, # of surrogates = 13, # of time steps = 20, # of temporal samples = 300)



        data_all.append(Pxx_matrix)
        raw_all.append(raw_matrix)







        ts_Pxx_seq, Pxx_seq_resp_sig, freq, _, _ = extract_windows(t_sig, resp_sig, Fs, windowing_params)
        label_all.append(Pxx_seq_resp_sig)
        
        
        # add meta

        _, HR_seq = extract_windows_signals(ts_Pxx, t_sig, df_resp['HR_cosmed'].values, Fs, windowing_params=windowing_params)
        HR_seq = HR_seq[:,windowing_params['seq_len']//2,:].mean(axis=1)
        
        _, VT_seq = extract_windows_signals(ts_Pxx, t_sig, df_resp['VT_cosmed'].values, Fs, windowing_params=windowing_params)
        VT_seq = VT_seq[:,windowing_params['seq_len']//2,:].mean(axis=1)
        
        _, RR_seq = extract_windows_signals(ts_Pxx, t_sig, df_resp['RR_cosmed'].values, Fs, windowing_params=windowing_params)
        RR_seq = RR_seq[:,windowing_params['seq_len']//2,:].mean(axis=1)
# HR_seq = HR_seq[:,windowing_params['seq_len']//2,:].mean(axis=1)
        
        
#         HR_seq = get_hr_seq(ts_Pxx_seq, t_sig, df_resp['HR_cosmed'].values).mean(axis=1)
#         VT_seq = get_hr_seq(ts_Pxx_seq, t_sig, df_resp['VT_cosmed'].values).mean(axis=1)
#         RR_seq = get_hr_seq(ts_Pxx_seq, t_sig, df_resp['RR_cosmed'].values).mean(axis=1)
        
        BMI = df_subinfo.loc[df_subinfo['Subject_ID']==subject_id, 'BMI'].values[0]
        
        meta_task = np.asarray([[int(subject_id), tasks_dict[task_name], BMI]]*Pxx_seq_resp_sig.shape[0])
        meta_task = np.c_[meta_task, HR_seq, VT_seq, RR_seq]
        meta_all.append(meta_task)
#         meta_all.append(np.asarray([[int(subject_id), tasks_dict[task_name], ]]*Pxx_seq_resp_sig.shape[0]))
        ts_all.append(ts_Pxx_seq)


print(Pxx_names)
print(freq)


label_all = np.concatenate(label_all,axis=0)
data_all = np.concatenate(data_all,axis=0)
raw_all = np.concatenate(raw_all,axis=0)
meta_all = np.concatenate(meta_all,axis=0)
ts_all = np.concatenate(ts_all,axis=0)
# hr_sub = np.concatenate(hr_sub,axis=0)

meta_names = ['subject_id', 'task_name', 'BMI', 'HR_cosmed', 'VT_cosmed', 'RR_cosmed']

print(label_all.shape, data_all.shape, raw_all.shape, meta_all.shape, ts_all.shape)
    
data_all =  float2uint8(data_all)
label_all =  float2uint8(label_all)

working on subject 101


SystemExit: 

/home/mchan/miniconda3/envs/mienv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [22]:
# df_resp_sub[df_resp_sub['task']=='Run']

In [ ]:
debug_get_sur_norm = False

if debug_get_sur_norm:

    norm_param = {
        'mode': 'medfilt',
        'length_medfilt': math.ceil(Fs * 4 / 2.) * 2 + 1,
    #     'length_medfilt': 51,
    }

    fig, ax = plt.subplots(1,1,figsize=(20,6), dpi=80)

#     ax.plot(t_sig, get_sur_norm(t_sig, sig, norm_param, debug=False))
#     ax.plot(t_sig, resp_sig+2)
#     plt.show()
    
    length_medfilt = norm_param['length_medfilt']
    sig_med = medfilt(sig, length_medfilt)

    sig_filt = sig - sig_med
    sig_norm = sig_filt/get_envelope(sig_filt)



    # hard coded, but it's generally ok
    threshold = 1.5
    indices = np.where((np.abs(sig_norm)>threshold))[0]
    sig_norm[indices] = np.sign(sig_norm[indices])*threshold

#     fig, ax = plt.subplots(1,1,figsize=(20,3), dpi=80)

    ax.plot(sig)
    ax.plot(sig_med+5)
    ax.plot(sig_filt+10)
    ax.plot(get_envelope(sig_filt)+15)
    ax.plot(sig_norm+20)
    
    
    ax.plot(resp_sig-5)

    plt.show()

# store stage dict

In [ ]:
stage3_dict = {
    'surrogate_names': surrogate_names,
    'meta_names': meta_names,
#     'list_output': list_output,
#     'list_meta': list_meta,
#     'list_signal': list_signal,
    'Fs': Fs,
    'subject_ids': np.unique(meta_all[:,0]),
    'task_ids': np.unique(meta_all[:,1]),
    'windowing_params': windowing_params,
    'freq': freq
}

data_saver(stage3_dict, 'stage3_dict', outputdir)

# export data

In [ ]:
data_all.shape, label_all.shape, meta_all.shape, raw_all.shape, ts_all.shape

In [ ]:
data_saver(data_all, 'data', outputdir)
# data_saver(feature_all, 'feature', outputdir)
data_saver(label_all, 'label', outputdir)
data_saver(meta_all, 'meta', outputdir)

data_saver(raw_all, 'raw', outputdir)
data_saver(ts_all, 'ts', outputdir)

In [ ]:
np.unique(meta_all[:,1])

In [ ]:
sys.exit()

In [ ]:
windowing_params

In [ ]:
meta_all.append([int(subject_id), tasks_dict[task_name]])
meta_all.append([int(subject_id), tasks_dict[task_name]])
meta_all

# get resp. signal segmented

In [ ]:
np.stack(raw_list, axis=3).transpose(0,3,1,2).shape

In [ ]:
np.stack(Pxx_list, axis=3).transpose(0,3,1,2).shape

In [ ]:
ts_sig_segments_seq.shape, sig_segments_seq.shape

In [ ]:
freq, Pxx = get_PSD_beats(sig_segments[0,:], Fs)


In [ ]:
Pxx.shape

In [ ]:
plt.plot(freq, Pxx)
plt.show()

In [ ]:
plt.plot(aaa[:,0])
plt.show()

In [ ]:
#  Pxx_seq_surrogate, freq
Pxx_seq_surrogate.shape
freq.shape

In [ ]:
plt.plot(freq, Pxx_seq_surrogate[0,0,:])
plt.show()

In [ ]:
def beat_segmentation(sig, i_peaks, start_offset=-50, end_offset=250):


In [ ]:
raw_all[1].shape

In [ ]:
np.stack(raw_all,axis=-1)

In [ ]:
# df_resp

In [ ]:
raw_matrix.shape

In [ ]:
raw_matrix.shape, df_resp['resp_cosmed'].values.shape

In [ ]:
# raw_list[0].shape

data_all.shape, raw_all.shape

In [ ]:
# len(data_all)
# data_all[0].shape

In [ ]:
# Pxx_seq_surrogate.shape



In [ ]:
Pxx_seq_surrogate.shape

# 4/14/2022 next, add the above block to the block below

In [ ]:
label_sub = []
data_sub = []
meta_sub = []
ts_sub = []
hr_sub = []



for subject_id in df_subjects_complete.keys():   
#     if subject_id!='CW010':
#         continue
    print('working on subject', subject_id)
        
    df = df_subjects_complete[subject_id]
    
    
    TASKS = np.unique(df_subjects_complete[subject_id]['task_id'])
    for task_id in TASKS:
        # 1. get df of a task
        df_task = df[df['task_id']==task_id] 
        
        if task_id == task_id_target: # focus on task 6 only
#         if task_id == 6: # focus on task 6 only

            br_sim_sig = df_task['br_sim_sig'].values
            br = df_task['br'].values
            t_sig = df_task['time'].values
            hr = df_task['heart_rate_cosmed'].values

            Pxx_list = []
            Pxx_names = []
            
            # method 1: convert to STFT
            for surrogate_name in modalities:

                surrogate_sig = df_task[surrogate_name].values
                
                ts_Pxx_seq, Pxx_seq_surrogate, freq = extract_windows(t_sig, surrogate_sig, Fs, norm_param)
                Pxx_list.append(Pxx_seq_surrogate)
                Pxx_names.append(surrogate_name)
                if debug_TF:
                    inspect_TF_rep(t_sig, surrogate_sig, br_sim_sig, norm_param, subject_id=subject_id, sig_name=surrogate_name, outputdir=outputdir)

#                 sys.exit()


            if windowing_params.get('remove_outliers'): 
                
                sqi_seq, threshold_SQI = extract_windows_sqi(t_sig, ts_Pxx_seq, ECG_SQI)
                
                # find data with high sqi
#                 indices_kept = np.where( (sqi_seq.min(axis=1)>threshold_SQI) & (sqi_seq.min(axis=1)>0.7) )[0]
                indices_kept = np.where( sqi_seq.min(axis=1)>0.65 )[0]
                print('\t{:.2f}% outliers removed, threshold={:.2f}'.format((1-indices_kept.shape[0]/sqi_seq.shape[0])*100, threshold_SQI))

#                 sys.exit()
            else:
                indices_kept = np.arange(ts_Pxx_seq.shape[0])
                

                
            # size of Pxx_matrix = (N_seq, N_modality, N_window, N_freq)
            Pxx_matrix = np.stack(Pxx_list, axis=3).transpose(0,3,1,2)

            ts_Pxx_seq, Pxx_seq_br_sim_sig, freq = extract_windows(t_sig, br_sim_sig, Fs)
            hr_seq = get_hr_seq(ts_Pxx_seq, t_sig, hr)

            # method 2: use ECG_AM and do min-max normalization on each column

            label_sub.append(Pxx_seq_br_sim_sig[indices_kept,:,:])
            data_sub.append(Pxx_matrix[indices_kept,:,:])
            hr_sub.append(hr_seq[indices_kept,:])

            meta_sub.append(np.ones(indices_kept.shape[0])*int(subject_id[-3:]))
            ts_sub.append(ts_Pxx_seq[indices_kept,:])

print(Pxx_names)
print(freq)



In [ ]:
data_saver(Pxx_names, 'Pxx_names', outputdir)
data_saver(freq, 'freq', outputdir)

In [ ]:
# br_sim_sig
# t_sig

In [ ]:
# t_interp = np.arange(t_sig[0]*Fs_new, t_sig[-1]*Fs_new+1)/Fs_new
# t_interp

In [ ]:
def extract_windows_single(t_sig_cosmed, t_sig, surrogate_sig, start_offset, end_offset, NFFT, noverlap):
    Pxx_seq = []
    ts_Pxx_seq = []
    for t in t_sig_cosmed:
        t_start = t + start_offset
        t_end = t + end_offset

        t_mask = (t_sig > t_start) & (t_sig <= t_end)

#         print('\t', t_start, t_end, surrogate_sig.shape, t_sig.shape, t_sig[0], t_sig[-1])
        
        
        sig = surrogate_sig[t_mask].copy()
        ts = t_sig[t_mask].copy()

        ts_Pxx, Pxx, freq = get_PSD_windows_scipy(ts, sig, Fs, NFFT, noverlap)
#         print('\t\t', surrogate_sig.shape, t_sig.shape, sig.shape)

        Pxx_seq.append(Pxx)
        ts_Pxx_seq.append(ts_Pxx)
#         print('\t\t\t', surrogate_sig.shape, t_sig.shape)


    # Pxx_seq.shape = (276, seq_len, N_feature)
    Pxx_seq = np.stack(Pxx_seq)
    # ts_Pxx_seq.shape = (276, N_feature, seq_len)
    ts_Pxx_seq = np.stack(ts_Pxx_seq)

    return ts_Pxx_seq, Pxx_seq, freq

In [ ]:
def get_padded_vectors(t_sig, surrogate_sig, Fs, sig_padded=60):
    """Summary or get_padded_vectors

    Parameters:
    t_sig (np arr): time vector of the signal
    surrogate_sig (np arr): value vector of the signal
    Fs (np arr): sampling rate
    sig_padded (int): duration (in s) of the signal padded at the begining and the end of the signal

    Returns:
    t_sig_padded: time vector of the padded signal
    surrogate_sig_padded: value vector of the padded signal

    """
#         sig_padded = 60
    offset = int(sig_padded*Fs) # reversely pad 10 sec to beginning and end of the signal
    # t_sig_padded = np.r_[t_sig[0:offset][::-1], t_sig, t_sig[-offset:][::-1]]

    t_end_padded = np.arange(offset)/Fs + 1/Fs
    t_end_padded = t_end_padded + t_sig[-1]

    t_start_padded = np.arange(offset)/Fs-sig_padded
    t_start_padded = t_start_padded + t_sig[0]

    t_sig_padded = np.r_[t_start_padded, t_sig, t_end_padded]

    surrogate_sig_padded = np.r_[surrogate_sig[0:offset][::-1], surrogate_sig, surrogate_sig[-offset:][::-1]]

    return t_sig_padded, surrogate_sig_padded

# contruct test dataset
## TODO: write comment

In [ ]:
label_sub_cosmed = []
data_sub_cosmed = []
meta_sub_cosmed = []
ts_sub_cosmed = []
hr_sub_cosmed = []
est_temporal_sub_cosmed = []
label_temporal_sub_cosmed = []

# length_medfilt = 2001
norm_param = {
    'mode': 'EMD',
}
norm_param = {
    'mode': 'medfilt',
    'length_medfilt': 2001,
}

window_size = windowing_params['window_size']

# this is pre-defined
start_offset = -window_size/2 - 3*10
end_offset = window_size/2 + 3*9

PADDING = True

NFFT = int(Fs*windowing_params['window_size'])  # 60 sec per window
noverlap = int(NFFT*windowing_params['overlap']) # int(60*0.95*Fs) = 57 sec * Fs

for subject_id in df_subjects_complete.keys():   
#     if subject_id!='CW001':
#         continue
    print('working on subject', subject_id)
        
    df = df_subjects_complete[subject_id]
    
    
    TASKS = np.unique(df_subjects_complete[subject_id]['task_id'])
    for task_id in TASKS:
        # 1. get df of a task
        df_task = df[df['task_id']==task_id] 
#         if task_id == 6: # focus on task 6 only
        if task_id == task_id_target: # focus on task 6 only

            br_sim_sig = df_task['br_sim_sig'].values
            br = df_task['br'].values
            hr = df_task['heart_rate_cosmed'].values

            Pxx_list = []
            Pxx_names = []

            # method 1: convert to STFT
            for surrogate_name in modalities:
                
                
                t_sig = df_task['time'].values

                
#                 print(surrogate_name)
                mask = (df_predictions['task_id']==task_id) & (df_predictions['subject_id']==subject_id) & (df_predictions['surrogate_name']==surrogate_name)

                df_predictions_sub = df_predictions[mask]
                indices_RR = df_predictions_sub['indices_RR'].values
                RR_est_coarse = df_predictions_sub['RR_est_coarse'].values
                RR_label_coarse = df_predictions_sub['RR_label_coarse'].values

                ts_cosmed = t_sig[indices_RR]
                
                surrogate_sig = df_task[surrogate_name].values

                if PADDING:
                    # print('padding 60s before and after')
                    t_sig_padded, surrogate_sig_padded = get_padded_vectors(t_sig, surrogate_sig, Fs, sig_padded=60)
                else:
                    print('not padding, need to modify the code')
                    break

#                 mask_valid_cosmed = (ts_cosmed + start_offset > t_sig[0]) & (ts_cosmed + end_offset < t_sig[-1])
                mask_valid_cosmed = (ts_cosmed + start_offset > t_sig_padded[0]) & (ts_cosmed + end_offset < t_sig_padded[-1])
                t_sig_cosmed = ts_cosmed[mask_valid_cosmed]
                
#                 print(t_sig.shape, surrogate_sig.shape)

                if norm_param is not None:
#                     surrogate_sig = get_sur_norm(t_sig, surrogate_sig, norm_param, debug=False)
                    surrogate_sig_padded = get_sur_norm(t_sig_padded, surrogate_sig_padded, norm_param, debug=False)

#                 print(t_sig.shape, surrogate_sig.shape)

#                 ts_Pxx_seq, Pxx_seq_surrogate, freq = extract_windows_single(t_sig_cosmed, t_sig, surrogate_sig, start_offset, end_offset, NFFT, noverlap)
                ts_Pxx_seq, Pxx_seq_surrogate, freq = extract_windows_single(t_sig_cosmed, t_sig_padded, surrogate_sig_padded, start_offset, end_offset, NFFT, noverlap)

                
#                 sys.exit()
                
                Pxx_list.append(Pxx_seq_surrogate)

            # size of Pxx_matrix = (N_seq, N_modality, N_window, N_freq)
            Pxx_matrix = np.stack(Pxx_list, axis=3).transpose(0,3,1,2)
            
#             print('br sig')
            
            
            if PADDING:
                # print('padding 60s before and after')
                t_sig_padded, br_sim_sig_padded = get_padded_vectors(t_sig, br_sim_sig, Fs, sig_padded=60)
                t_sig_padded, hr_padded = get_padded_vectors(t_sig, hr, Fs, sig_padded=60)

            ts_Pxx_seq, Pxx_seq_br_sim_sig, freq = extract_windows_single(t_sig_cosmed, t_sig_padded, br_sim_sig_padded, start_offset, end_offset, NFFT, noverlap)
            hr_seq = get_hr_seq(ts_Pxx_seq, t_sig_padded, hr_padded)
#             ts_Pxx_seq, Pxx_seq_br_sim_sig, freq = extract_windows_single(t_sig_cosmed, t_sig, br_sim_sig, start_offset, end_offset, NFFT, noverlap)
#             hr_seq = get_hr_seq(ts_Pxx_seq, t_sig, hr)

            # method 2: use ECG_AM and do min-max normalization on each column
#             ts_Pxx_seq, Pxx_seq_ECG_AM, freq = extract_windows(ECG_AM, Fs)

            label_sub_cosmed.append(Pxx_seq_br_sim_sig)
            data_sub_cosmed.append(Pxx_matrix)

            est_temporal_sub_cosmed.append(RR_est_coarse[mask_valid_cosmed])
            label_temporal_sub_cosmed.append(RR_label_coarse[mask_valid_cosmed])
            meta_sub_cosmed.append(np.ones(ts_Pxx_seq.shape[0])*int(subject_id[-3:]))
            ts_sub_cosmed.append(ts_Pxx_seq)
            hr_sub_cosmed.append(hr_seq)

            
#             print(ts_sub_cosmed[-1].shape, meta_sub_cosmed[-1].shape, est_temporal_sub_cosmed[-1].shape, data_sub_cosmed[-1].shape, label_sub_cosmed[-1].shape, label_temporal_sub_cosmed[-1].shape)
#             sys.exit()
            
# print(Pxx_names)
# print(freq)

# data_saver(Pxx_names, 'Pxx_names', outputdir)
# data_saver(freq, 'freq', outputdir)

In [ ]:
# (M,N) = ts_Pxx_seq.shape

# hr_seq = np.zeros((M,N))

# for i in range(M):
#     for j in range(N):
#         index = np.where(np.around(t_sig_padded, decimals=3)==np.around(ts_Pxx_seq[i,j], decimals=3))[0]
#         print(index, ts_Pxx_seq[i,j], t_sig_padded)
#         hr_seq[i,j] = hr[index]
# # return hr_seq

In [ ]:
# np.where(t_sig_padded==ts_Pxx_seq[i,j])

In [ ]:
ts_Pxx_seq.shape, t_sig_padded.shape, hr_padded.shape

In [ ]:
# t_sig_padded

In [ ]:
# t_sig_cosmed, t_sig, surrogate_sig

In [ ]:
# t_sig.shape, surrogate_sig.shape

In [ ]:
# t_sig_cosmed.shape, t_sig.shape, surrogate_sig.shape

In [ ]:
# ts_cosmed.shape

In [ ]:
# mask_valid_cosmed = (ts_cosmed + start_offset > t_sig[0]) & (ts_cosmed + end_offset < t_sig[-1])
# mask_valid_cosmed

In [ ]:
# ts_cosmed + start_offset

In [ ]:
# ts_cosmed

In [ ]:
# t_sig_cosmed

# TODO: remove this fat ass cell

In [ ]:
plot_Pxx_testing = False
if plot_Pxx_testing == True:
    inspect_TF_rep(t_sig, ECG_AM, br_sim_sig)


In [ ]:
plot_Pxx_testing = False
if plot_Pxx_testing == True:
    inspect_TF_rep(t_sig, SCG_AMpt, br_sim_sig)

In [ ]:
plot_Pxx_testing = False
if plot_Pxx_testing == True:
    inspect_TF_rep(t_sig, SCG_AM_norm, br_sim_sig)

In [ ]:
# ECG_AMpt_norm = get_sur_norm(ECG_AMpt, length_medfilt=2001)
# SCG_AM_norm = get_sur_norm2(t_sig, SCG_AM, norm_param, debug=False)

plot_Pxx_testing = False
if plot_Pxx_testing == True:
    inspect_TF_rep(t_sig, SCG_AM_norm, br_sim_sig)


In [ ]:
label_sub = np.concatenate(label_sub,axis=0)
data_sub = np.concatenate(data_sub,axis=0)
meta_sub = np.concatenate(meta_sub,axis=0)
ts_sub = np.concatenate(ts_sub,axis=0)
hr_sub = np.concatenate(hr_sub,axis=0)

data_sub =  float2uint8(data_sub)
label_sub =  float2uint8(label_sub)

In [ ]:
label_sub.shape, data_sub.shape, meta_sub.shape, ts_sub.shape, hr_sub.shape

In [ ]:
label_sub_cosmed = np.concatenate(label_sub_cosmed,axis=0)
data_sub_cosmed = np.concatenate(data_sub_cosmed,axis=0)
meta_sub_cosmed = np.concatenate(meta_sub_cosmed,axis=0)
ts_sub_cosmed = np.concatenate(ts_sub_cosmed,axis=0)
hr_sub_cosmed = np.concatenate(hr_sub_cosmed,axis=0)

est_temporal_sub_cosmed = np.concatenate(est_temporal_sub_cosmed,axis=0)
label_temporal_sub_cosmed = np.concatenate(label_temporal_sub_cosmed,axis=0)

data_sub_cosmed =  float2uint8(data_sub_cosmed)
label_sub_cosmed =  float2uint8(label_sub_cosmed)

In [ ]:
hr_sub_cosmed.shape, ts_sub_cosmed.shape, meta_sub_cosmed.shape, est_temporal_sub_cosmed.shape, data_sub_cosmed.shape, label_sub_cosmed.shape, label_temporal_sub_cosmed.shape

In [ ]:
# ts_sub_cosmed[:,0]

TODO: split data, implement U-Net

# split data

In [ ]:
def get_indices_set(ids_arr, data_arr):
    indices_set = []
    for id_sub in ids_arr:
        indices_set.append(np.where(data_arr==id_sub)[0])
    return np.concatenate(indices_set)

In [ ]:
def split_matrix(data, label, meta, ts, hr, indices_set, est_temporal=None, label_temporal=None, set_name='null'):

    data = data[indices_set,:,:,:]
    label = label[indices_set,:,:]
    meta = meta[indices_set]
    ts = ts[indices_set, :]
    hr = hr[indices_set, :]
    
    if (est_temporal is not None) & (label_temporal is not None):
        est_temporal = est_temporal[indices_set]
        label_temporal = label_temporal[indices_set]
        return data, label, meta, ts, hr, est_temporal, label_temporal
    else:
        return data, label, meta, ts, hr

def save_matrix(data, label, meta, ts, hr, outputdir, est_temporal=None, label_temporal=None, set_name='null'):
    outputdir_set = outputdir+set_name+'/'

    if not os.path.exists(outputdir_set):
        os.makedirs(outputdir_set)

    data_saver(data, 'data', outputdir_set)
    data_saver(label, 'label', outputdir_set)
    data_saver(meta, 'meta', outputdir_set)
    data_saver(ts, 'ts', outputdir_set)
    data_saver(hr, 'hr', outputdir_set)
    
    if (est_temporal is not None) & (label_temporal is not None):
        data_saver(est_temporal, 'est_temporal', outputdir_set)
        data_saver(label_temporal, 'label_temporal', outputdir_set)
    


# LOSO CV

In [ ]:
# ids_all = np.unique(meta_sub)
# kfold = ids_all.shape[0]
# kf = KFold(n_splits=kfold, shuffle=True, random_state=i_seed)
# kf.get_n_splits(ids_all)
# print(kf)  

# for i_CV, (train_idx, val_idx) in enumerate(kf.split(ids_all)):
    
#     outputdir_CV = outputdir+'CV{}/'.format(i_CV)
#     print('index CV', i_CV)
#     print('\tSub IDs | TRAIN:', np.sort(ids_all[train_idx]), 'VAL:', np.sort(ids_all[val_idx]))

    
    
#     ids_train = ids_all[train_idx]
#     ids_val = ids_all[val_idx]
    
    
#     # interpolated datapoints
#     indices_TRAIN = get_indices_set(ids_train, meta_sub)
#     indices_TEST = get_indices_set(ids_val, meta_sub)
    
#     data_TRAIN, label_TRAIN, meta_TRAIN, ts_TRAIN, hr_TRAIN = split_matrix(data_sub, label_sub, meta_sub, ts_sub, hr_sub, indices_TRAIN, set_name='TRAIN')
#     save_matrix(data_TRAIN, label_TRAIN, meta_TRAIN, ts_TRAIN, hr_TRAIN, outputdir_CV, set_name='TRAIN')
    
#     data_TEST, label_TEST, meta_TEST, ts_TEST, hr_TEST = split_matrix(data_sub, label_sub, meta_sub, ts_sub, hr_sub, indices_TEST, set_name='TEST')
#     save_matrix(data_TEST, label_TEST, meta_TEST, ts_TEST, hr_TEST, outputdir_CV, set_name='TEST')

    

#     indices_TRAINc = get_indices_set(ids_train, meta_sub_cosmed)
#     indices_TESTc = get_indices_set(ids_val, meta_sub_cosmed)

#     data_TRAINc, label_TRAINc, meta_TRAINc, ts_TRAINc, hr_TRAINc, est_temporal_TRAINc, label_temporal_TRAINc = split_matrix(data_sub_cosmed, label_sub_cosmed, meta_sub_cosmed, ts_sub_cosmed, hr_sub_cosmed, indices_TRAINc, est_temporal_sub_cosmed, label_temporal_sub_cosmed, set_name='TRAIN_cosmed')
#     save_matrix(data_TRAINc, label_TRAINc, meta_TRAINc, ts_TRAINc, hr_TRAINc, outputdir_CV, est_temporal_TRAINc, label_temporal_TRAINc, set_name='TRAIN_cosmed')

#     data_TESTc, label_TESTc, meta_TESTc, ts_TESTc, hr_TESTc, est_temporal_TESTc, label_temporal_TESTc = split_matrix(data_sub_cosmed, label_sub_cosmed, meta_sub_cosmed, ts_sub_cosmed, hr_sub_cosmed, indices_TESTc, est_temporal_sub_cosmed, label_temporal_sub_cosmed, set_name='TEST_cosmed')
#     save_matrix(data_TESTc, label_TESTc, meta_TESTc, ts_TESTc, hr_TESTc, outputdir_CV, est_temporal_TESTc, label_temporal_TESTc, set_name='TEST_cosmed')

    
#     data_saver(Pxx_names, 'Pxx_names', outputdir_CV)
#     data_saver(freq, 'freq', outputdir_CV)
# #     save_matrix(data_train, label_train, meta_train, ts_train, hr_train, outputdir+'CV{}/'.format(i_rep,i_CV), set_name='train')
# #     save_matrix(data_val, label_val, meta_val, ts_val, hr_val, outputdir+'CV{}/'.format(i_rep,i_CV), set_name='val')

In [ ]:
# indices_TRAIN

In [ ]:
ids_all = np.unique(meta_sub)
N_sub = ids_all.shape[0]
# np.shuffle(i_sub_unique_all)

np.random.seed(i_seed)
np.random.shuffle(ids_all)
# i_sub_unique_all

ids_TRAIN = ids_all[:int(N_sub*.7)]
# ids_val = ids_train[int(ids_train.shape[0]*.7):]
# ids_train = ids_train[:int(ids_train.shape[0]*.7)]


ids_TEST = ids_all[int(N_sub*.7):]

# ids_TRAIN, ids_TEST

# interpolated datapoints
indices_TRAIN = get_indices_set(ids_TRAIN, meta_sub)
indices_TEST = get_indices_set(ids_TEST, meta_sub)
data_TRAIN, label_TRAIN, meta_TRAIN, ts_TRAIN, hr_TRAIN = split_matrix(data_sub, label_sub, meta_sub, ts_sub, hr_sub, indices_TRAIN, set_name='TRAIN')
save_matrix(data_TRAIN, label_TRAIN, meta_TRAIN, ts_TRAIN, hr_TRAIN, outputdir, set_name='TRAIN')

data_TEST, label_TEST, meta_TEST, ts_TEST, hr_TEST = split_matrix(data_sub, label_sub, meta_sub, ts_sub, hr_sub, indices_TEST, set_name='TEST')
save_matrix(data_TEST, label_TEST, meta_TEST, ts_TEST, hr_TEST, outputdir, set_name='TEST')



# COSMED datapoints
# indices_TEST = get_indices_set(ids_TEST, meta_sub)
indices_TRAINc= get_indices_set(ids_TRAIN, meta_sub_cosmed)
indices_TESTc = get_indices_set(ids_TEST, meta_sub_cosmed)

data_TRAINc, label_TRAINc, meta_TRAINc, ts_TRAINc, hr_TRAINc, est_temporal_TRAINc, label_temporal_TRAINc = split_matrix(data_sub_cosmed, label_sub_cosmed, meta_sub_cosmed, ts_sub_cosmed, hr_sub_cosmed, indices_TRAINc, est_temporal_sub_cosmed, label_temporal_sub_cosmed, set_name='TRAIN_cosmed')
save_matrix(data_TRAINc, label_TRAINc, meta_TRAINc, ts_TRAINc, hr_TRAINc, outputdir, est_temporal_TRAINc, label_temporal_TRAINc, set_name='TRAIN_cosmed')

data_TESTc, label_TESTc, meta_TESTc, ts_TESTc, hr_TESTc, est_temporal_TESTc, label_temporal_TESTc = split_matrix(data_sub_cosmed, label_sub_cosmed, meta_sub_cosmed, ts_sub_cosmed, hr_sub_cosmed, indices_TESTc, est_temporal_sub_cosmed, label_temporal_sub_cosmed, set_name='TEST_cosmed')
save_matrix(data_TESTc, label_TESTc, meta_TESTc, ts_TESTc, hr_TESTc, outputdir, est_temporal_TESTc, label_temporal_TESTc, set_name='TEST_cosmed')


# 5fold cv

In [ ]:
# data_TRAIN.shape, data_TEST.shape

In [ ]:

from sklearn.model_selection import KFold

CV_n = 5
rep_n = 5

kfold = CV_n

print('all ids_TRAIN', ids_TRAIN)
print('all ids_TEST', ids_TEST)
print('will save to', outputdir)
for i_rep in range(rep_n):

    print('\n\n')
    kf = KFold(n_splits=kfold, shuffle=True, random_state=i_seed+i_rep)
    kf.get_n_splits(ids_TRAIN)
    print(kf)  

    for i_CV, (train_idx, val_idx) in enumerate(kf.split(ids_TRAIN)):
        print('----------------Splitting for rep {}, CV {}----------------'.format(i_rep, i_CV))
        print("Sub ID | TRAIN:", np.sort(ids_TRAIN[train_idx]), "VAL:", np.sort(ids_TRAIN[val_idx]))
        print('index CV', CV_n*i_rep+i_CV)

        ids_train = ids_TRAIN[train_idx]
        ids_val = ids_TRAIN[val_idx]
        
        indices_train = get_indices_set(ids_train, meta_sub)
        indices_val = get_indices_set(ids_val, meta_sub)
    
        data_train, label_train, meta_train, ts_train, hr_train = split_matrix(data_sub, label_sub, meta_sub, ts_sub, hr_sub, indices_train, set_name='train')
        data_val, label_val, meta_val, ts_val, hr_val = split_matrix(data_sub, label_sub, meta_sub, ts_sub, hr_sub, indices_val, set_name='val')

        save_matrix(data_train, label_train, meta_train, ts_train, hr_train, outputdir+'rep{}/CV{}/'.format(i_rep,i_CV), set_name='train')
        save_matrix(data_val, label_val, meta_val, ts_val, hr_val, outputdir+'rep{}/CV{}/'.format(i_rep,i_CV), set_name='val')



# TBD

# ======================== TEST ========================

# HR and RR trade-off

## TBD (already in stage1)

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# RR = np.linspace(label_range_dict['br'][0], label_range_dict['br'][1], 100)
# HR = np.linspace(label_range_dict['heart_rate_cosmed'][0], label_range_dict['heart_rate_cosmed'][1], 100)

# fig, (ax1) = plt.subplots(1, 1, figsize=(5,5), dpi=120)
# fontsize=15

# ax1.fill_between(HR, 0, RR, alpha=0.5, label='feasible region')
# ax1.fill_between(HR, RR[-1], RR, color='gray', alpha=0.1, label='infeasible region')
# ax1.set_title('feasible "Nyquist region" to derive RR using EDR', fontsize=fontsize)

# ax1.set_ylabel(r'$F_{signal}$ RR (bpm)', fontsize=fontsize)
# ax1.set_xlabel(r'$F_{sampling}$ HR (bpm)', fontsize=fontsize)
# ax1.set_xlim(HR[0], HR[-1])
# ax1.set_ylim(RR[0], RR[-1])
# ax1.legend(frameon=True, loc='upper left')


# CWT research (WIP)

In [ ]:

# import pywt
# import numpy as np
# import matplotlib.pyplot as plt
# x = np.arange(512)
# y = np.sin(2*np.pi*x/32)
# coef, freqs=pywt.cwt(y,np.arange(1,129),'gaus1')

# fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20,5))
# ax1.plot(x,y)
# ax1.set_xlim(x[0], x[-1])


# ax2.imshow(coef, extent=[x[0], x[-1], 1, 129]) # doctest: +SKIP


In [ ]:
# import pywt
# import numpy as np
# import matplotlib.pyplot as plt
# t = np.linspace(-1, 1, 200, endpoint=False)
# sig  = np.cos(2 * np.pi * 7 * t) + np.real(np.exp(-7*(t-0.4)**2)*np.exp(1j*2*np.pi*2*(t-0.4)))
# widths = np.arange(1, 31)
# cwtmatr, freqs = pywt.cwt(sig, widths, 'mexh')

# fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20,5))
# ax1.plot(t,sig)
# ax1.set_xlim(t[0], t[-1])
# ax2.imshow(cwtmatr, extent=[-1, 1, 1, 31], cmap='PRGn', aspect='auto',
#             vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())  # doctest: +SKIP


In [ ]:
# # [-offset:]
# offset = 100000
# wavelet_name = 'gaus8'
# wavlist = pywt.wavelist(kind='continuous')
# wavlist

In [ ]:
# import pywt
# from pywt import scale2frequency
# import numpy as np
# import matplotlib.pyplot as plt
# t = df_task['time'].values[-offset:]
# # sig  = ECG_AM_norm[-offset:]
# sig  = ECG_AM[-offset:]
# # sig = br_sim_sig
# widths = np.arange(1, 31)
# cwtmatr, freqs = pywt.cwt(sig, widths, wavelet_name)
# f = scale2frequency(wavelet_name, widths)

# fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(20,5))
# ax1.plot(t,sig)
# ax1.set_xlim(t[0], t[-1])
# # ax2.imshow(cwtmatr, extent=[-1, 1, 1, 31], cmap='PRGn', aspect='auto',
# #             vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())  # doctest: +SKIP

# # ax2.imshow(cwtmatr, extent=[-1, 1, f[0], f[-1]], cmap='viridis', aspect='auto',
# #             vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())  # doctest: +SKIP
# ax2.imshow(cwtmatr, extent=[-1, 1, f[0], f[-1]], cmap='viridis', aspect='auto')
# ax3.plot(t[50:-50], cwtmatr[5,50:-50])


In [ ]:
# import pywt
# from pywt import scale2frequency
# import numpy as np
# import matplotlib.pyplot as plt
# t = df_task['time'].values[-offset:]
# # sig  = ECG_AM_norm
# sig = br_sim_sig[-offset:]
# widths = np.arange(1, 31)
# cwtmatr, freqs = pywt.cwt(sig, widths, wavelet_name)
# f = scale2frequency(wavelet_name, widths)/Fs 

# fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(20,5))
# ax1.plot(t,sig)
# ax1.set_xlim(t[0], t[-1])
# # ax2.imshow(cwtmatr, extent=[-1, 1, 1, 31], cmap='PRGn', aspect='auto',
# #             vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())  # doctest: +SKIP

# ax2.imshow(cwtmatr, extent=[-1, 1, f[0], f[-1]], cmap='viridis', aspect='auto',
#             vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())  # doctest: +SKIP

# ax3.plot(t[50:-50], cwtmatr[1,50:-50])


In [ ]:
# sig = br_sim_sig[0:int(100*Fs)]
# aaa = scipy.fftpack.fft(sig)

# T = 1/Fs
# N = sig.shape[0]
# xf = np.linspace(0.0, 1.0/(2.0*T), int(N/2)+1)

# xf

In [ ]:
# from scipy import signal
# import matplotlib.pyplot as plt
# # Generate a test signal, a 2 Vrms sine wave at 1234 Hz, corrupted by 0.001 V**2/Hz of white noise sampled at 10 kHz.

# fs = 10e3
# N = 1e5
# amp = 2*np.sqrt(2)
# freq = 1234.0
# noise_power = 0.001 * fs / 2
# time = np.arange(N) / fs
# x = amp*np.sin(2*np.pi*freq*time)
# x += np.random.normal(scale=np.sqrt(noise_power), size=time.shape)


# f, Pxx_den = signal.welch(x, fs, nperseg=1024)
# plt.semilogy(f, Pxx_den)
# plt.ylim([0.5e-3, 1])
# plt.xlabel('frequency [Hz]')
# plt.ylabel('PSD [V**2/Hz]')
# plt.show()

In [ ]:
# np.mean(Pxx_den[256:])


In [ ]:

# f, Pxx_spec = signal.welch(x, fs, 'flattop', 1024, scaling='spectrum')
# plt.figure()
# plt.semilogy(f, np.sqrt(Pxx_spec))
# plt.xlabel('frequency [Hz]')
# plt.ylabel('Linear spectrum [V RMS]')
# plt.show()
# # The peak height in the power spectrum is an estimate of the RMS amplitude.

# np.sqrt(Pxx_spec.max())
# # 2.0077340678640727

# ======================== TEST ========================

## check normalization
TODO: make these functions

In [ ]:

# # Pxx, freqs, bins = plot_sig_spectrogram(br_sim_sig, br, Fs, cmap=cmap)
# # indices = Pxx.argmax(axis=0)
# # br_arr = freqs[indices]*60 
# # print(br_arr.max(), br_arr.min())

# # Pxx, freqs, bins = plot_sig_spectrogram(ECG_AM_filt, br, Fs, cmap=cmap)

# # indices = Pxx.argmax(axis=0)
# # br_arr = freqs[indices]*60 
# # print(br_arr.max(), br_arr.min())


# # Pxx, freqs, bins = plot_sig_spectrogram(SCG_AM_filt, br, Fs, cmap=cmap)
# # indices = Pxx.argmax(axis=0)
# # br_arr = freqs[indices]*60 
# # print(br_arr.max(), br_arr.min())

# ####### sig normalized (+1/-1)

# print(br.max(), br.min())

# Pxx1, freqs, bins = plot_sig_spectrogram(br_sim_sig, br, Fs, cmap=cmap)
# indices = Pxx1.argmax(axis=0)
# br_arr1 = freqs[indices]*60 
# print(br_arr1.max(), br_arr1.min())

# Pxx2, freqs, bins = plot_sig_spectrogram(ECG_AM_norm, br, Fs, cmap=cmap)

# indices = Pxx2.argmax(axis=0)
# br_arr2 = freqs[indices]*60 
# print(br_arr2.max(), br_arr2.min())


# # Pxx3, freqs, bins = plot_sig_spectrogram(SCG_AM_norm, br, Fs, cmap=cmap)
# # indices = Pxx3.argmax(axis=0)
# # br_arr3 = freqs[indices]*60 
# # print(br_arr3.max(), br_arr3.min())


check xcorr

In [ ]:
# def lag_finder(y1, y2, sr):
#     n = len(y1)

#     corr = signal.correlate(y2, y1, mode='same') / np.sqrt(signal.correlate(y1, y1, mode='same')[int(n/2)] * signal.correlate(y2, y2, mode='same')[int(n/2)])

#     delay_arr = np.linspace(-0.5*n/sr, 0.5*n/sr, n)
#     delay = delay_arr[np.argmax(corr)]
#     print('y2 is ' + str(delay) + ' behind y1')

#     plt.figure()
#     plt.plot(delay_arr, corr)
#     plt.title('Lag: ' + str(np.round(delay, 3)) + ' s')
#     plt.xlabel('Lag')
#     plt.ylabel('Correlation coeff')
#     plt.show()
#     return delay

# t = np.arange(br_sim_sig.shape[0])/Fs


# shift = lag_finder(br_sim_sig, ECG_AM_norm, sr=Fs)


# figure, (ax0) = plt.subplots(1, 1, figsize=(40,2))

# ax0.plot(t, br_sim_sig, 'blue')

# ax0.plot(t, ECG_AM_norm, 'red', alpha=0.5)

# ax0.scatter(t+shift, ECG_AM_norm, c='k',s=0.01, alpha=0.8)
# ax0.set_xlabel('time (sec)')


# another approach

In [ ]:
# column_names = ['time', 'ECG', 'accelX', 'accelY', 'accelZ', 
#                 'br', 'br_sim_sig', 
#                 'ECG_AM', 'ECG_FM', 'ECG_BW', 
#                 'SCG_AM', 'SCG_FM', 'SCG_BW', 
#                 'PEP_FM', 
#                 'task_id', 'subject_id'
#                ]

# indices_data = [1,2,3,4,7,10,13]
# indices_data = [1,4]
# indices_label = [5,6]
# indices_meta = [14,15]

## windowing

In [ ]:
# buffer_dur = 5 # sec, don't use the first and last 5 seconds
# window_dur = 8 # sec, please refer to the RespNet paper
# window_size = int(window_dur*Fs)

# overlap_perc = 0.5

# window_shift = int((1-overlap_perc)*window_size)
        
    
# data_windows = []

# for subject_id in df_subjects.keys():   
#     print('working on subject', subject_id)
    
#     df = df_subjects[subject_id]    

    
#     TASKS = np.unique(df_subjects[subject_id]['task_id'])
#     for task_id in TASKS:
#         df_task = df[df['task_id']==task_id] 
        
#         data_matrix = df_task[column_names].values
#         data_matrix = data_matrix[buffer_dur*Fs:-buffer_dur*Fs]



#         N_window = math.floor((data_matrix.shape[0]-window_size)/window_shift)+1 # the amount of epochs there should be

#         indexer = np.arange(window_size)[None, :] + window_shift*np.arange(N_window)[:, None]
#         data_matrix = data_matrix[indexer,:]
# #         sys.exit()
#         # TODO: move downsample to stage2 so I can inspect the effect of downsampling
# #         data_matrix = data_matrix[:,::10,:]
#         data_windows.append(data_matrix)
        
# data_windows = np.concatenate(data_windows, axis=0)

In [ ]:
# ids_all = np.unique(data_windows[:,0,-1])
# N_sub = ids_all.shape[0]
# # np.shuffle(i_sub_unique_all)

# np.random.seed(i_seed)
# np.random.shuffle(ids_all)
# # i_sub_unique_all

# ids_train = ids_all[:int(N_sub*.7)]
# ids_val = ids_train[int(ids_train.shape[0]*.7):]
# ids_train = ids_train[:int(ids_train.shape[0]*.7)]


# ids_test = ids_all[int(N_sub*.7):]

In [ ]:
# ids_train, ids_val, ids_test

In [ ]:
# def get_indices_set(ids_arr, data_arr):
#     indices_set = []
#     for id_sub in ids_arr:
#         indices_set.append(np.where(data_arr==id_sub)[0])
#     return np.concatenate(indices_set)



# indices_train = get_indices_set(ids_train, data_windows[:,0,-1])
# indices_val = get_indices_set(ids_val, data_windows[:,0,-1])
# indices_test = get_indices_set(ids_test, data_windows[:,0,-1])


In [ ]:
# def split_matrix(data_windows, indices_set, set_name='null'):

#     data = data_windows[:,:,indices_data][indices_set,:,:]
#     label = data_windows[:,:,indices_label][indices_set,:,:]
#     meta = data_windows[:,:,indices_meta][indices_set,0,:]
#     return data, label, meta

# def save_matrix(data, label, meta, outputdir, set_name='null'):
#     outputdir_set = outputdir+set_name+'/'

#     if not os.path.exists(outputdir_set):
#         os.makedirs(outputdir_set)

#     data_saver(data, 'data', outputdir_set)
#     data_saver(label, 'label', outputdir_set)
#     data_saver(meta, 'meta', outputdir_set)
    
    


# normalization (0 mean unit variance)

In [ ]:
# data_train, label_train, meta_train = split_matrix(data_windows, indices_train, set_name='train')
# data_val, label_val, meta_val = split_matrix(data_windows, indices_val, set_name='val')
# data_test, label_test, meta_test = split_matrix(data_windows, indices_test, set_name='test')

# data_mean = data_train.mean()
# data_std = data_train.std()

# data_train = (data_train - data_mean) / data_std
# data_val = (data_val - data_mean) / data_std
# data_test = (data_test - data_mean) / data_std

# save data

In [ ]:
# save_matrix(data_train, label_train, meta_train, outputdir, set_name='train')
# save_matrix(data_val, label_val, meta_val, outputdir, set_name='val')
# save_matrix(data_test, label_test, meta_test, outputdir, set_name='test')


In [ ]:


# for axis in range(aaa.shape[2]):
#     fig=plt.figure(figsize=(5, 5), dpi= 80, facecolor='w', edgecolor='k')
#     ax = fig.add_subplot(1, 1, 1)
#     ax.hist(aaa.reshape(-1), 200, alpha=0.5, label='axis{}'.format(axis))
#     ax.legend( fontsize = 15 )
#     ax.set_ylabel('count', fontsize = 15)
#     ax.set_xlabel('raw value (a.u.)', fontsize = 15)
#     ax.set_title('raw value distribution', fontsize = 20)

#     ax.set_xlim([np.min(aaa),np.max(aaa)])

#     fig.savefig(outputdir + 'raw_distribution.png')

In [ ]:
# data_all.shape, actlabels_all.shape, sub_all.shape, DataNameList_idx_all.shape

# data_saver(data_all, 'data', outputdir_train)
# data_saver(actlabels_all, 'labels', outputdir_train)
# data_saver(sub_all, 'i_sub', outputdir_train)

In [ ]:


# # for set_name in ['train', 'val', 'test']:

# def save_data_ML()
#     outputdir_set = outputdir+set_name+'/'

#     if not os.path.exists(outputdir_set):
#         os.makedirs(outputdir_set)

#     data_saver(data, 'data', outputdir_set)
#     data_saver(label, 'label', outputdir_set)
#     data_saver(meta, 'meta', outputdir_set)


# TBD

In [ ]:
# from sklearn.metrics import mean_squared_error


# print('{}\t{}\t{}\t{}\t\t{}'.format('sig', 'MSE_peaks', 'MSE_valleys', 'xcoeff', 'delay'))


# for i, surrogate_name in enumerate(surrogate_dict.keys()):

#     sig_mod = surrogate_dict[surrogate_name]['normed']


#     RR_peaks, RR_valleys = get_RR_interp(ts, sig_mod)
    
#     r = np.corrcoef(RR_label, RR_peaks)
    
    
#     RMSE_peaks = np.sqrt(mean_squared_error(RR_label[indices_RR], RR_peaks[indices_RR]))
#     RMSE_valleys = np.sqrt(mean_squared_error(RR_label[indices_RR], RR_valleys[indices_RR]))
#     xcoeff = np.corrcoef( sig_mod, br_sim_sig )[0,1]
    
# #     fig = plt.figure(figsize=(12,4), dpi=60)
# #     ax = fig.add_subplot(111)
# #     ax.plot(t_raw, br_sim_sig, 'steelblue')
# #     ax.plot(t_raw, sig_mod, 'r')

#     npts = sig_mod.shape[0]
#     lags = np.arange(-npts + 1, npts)

#     ccov = np.correlate(sig_mod - sig_mod.mean(), br_sim_sig_norm - br_sim_sig_norm.mean(), mode='full')
#     ccor = ccov / (npts * sig_mod.std() * br_sim_sig_norm.std())

#     fig = plt.figure(figsize=(12,4), dpi=60)

# #     fig, axs = plt.subplots(nrows=2, dpi=60)
#     fig.subplots_adjust(hspace=0.4)
#     ax = fig.add_subplot(2,1,1)
#     ax.plot(t_raw, sig_mod, 'b', label='sig_mod')
#     ax.plot(t_raw, br_sim_sig_norm, 'r', label='br_sim')
#     ax.set_ylim(-10, 10)
#     ax.legend(loc='upper right', fontsize='small', ncol=2)

#     ax = fig.add_subplot(2,1,2)
#     ax.plot(lags/1000, ccor)

#     i = ccor.argmax()
#     ax.scatter((i-npts)/1000, ccor[i])
    
#     delay = (i-npts)/1000
    
#     print('{}\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{:.2f}'.format(surrogate_name, RMSE_peaks, RMSE_valleys, xcoeff, delay))
# #     sys.exit()

In [ ]:



# for i, acc_name in enumerate(['accelX', 'accelY', 'accelZ']):
#     sig_beats = scg_beats[acc_name]

#     # TODO: replace this with SQI based outlier removal
#     indices_CI = arg_CI(np.var(sig_beats,axis=0), confidence_interv=90)
#     print(indices_CI.shape)
#     sig_beats = sig_beats[:,indices_CI]
    
    
#     (i_aos, data_aos) = i_fiducials[acc_name]

#     ax = fig.add_subplot(3, 1, i+1)
#     plot_beats(ax, sig_beats, i_fiducials=(i_aos[indices_CI], data_aos[indices_CI]), Fs=FS_SCG, sig_name='[{}, {}] - {}'.format(subject_id, task_name, acc_name), color='steelblue')


In [ ]:
# fig = plt.figure(figsize=(4,12), dpi=80)

# for i, acc_name in enumerate(['accelX', 'accelY', 'accelZ']):
#     sig_beats = scg_beats[acc_name]

#     # TODO: replace this with SQI based outlier removal
#     indices_CI = arg_CI(np.var(sig_beats,axis=0), confidence_interv=90)
#     print(indices_CI.shape)
#     sig_beats = sig_beats[:,indices_CI]
    
    
#     (i_aos, data_aos) = i_fiducials[acc_name]

#     ax = fig.add_subplot(3, 1, i+1)
#     plot_beats(ax, sig_beats, i_fiducials=(i_aos[indices_CI], data_aos[indices_CI]), Fs=FS_SCG, sig_name='[{}, {}] - {}'.format(subject_id, task_name, acc_name), color='steelblue')

# fig.tight_layout()



# inspect patch signal

In [ ]:
# # don't really need this (TBD)
# def plot_sig_spectrogram(sig, FS):
#     x = sig
#     N = x.shape[0]
#     t_sig = np.linspace(0, (N-1)/FS, N, endpoint=False)

#     f, t, Sxx = signal.spectrogram(x, FS, nperseg=128)

#     figure, (ax_raw, ax_spectrogram) = plt.subplots(2, 1, figsize=(20,3))

#     ax_raw.plot(t_sig, x, 'red')
#     ax_raw.set_xlim([0,t_sig[-1]])

#     fontdict = {'size': 10}

#     NFFT = int(FS*0.5)  # 5ms window
#     noverlap = int(FS*0.5)

#     Pxx, freqs, bins, im = ax_spectrogram.specgram(x,Fs=FS, NFFT=NFFT)
#     ax_spectrogram.set_xlim([0,t_sig[-1]])

#     ax_spectrogram.set_xlabel('time (sec)', fontdict=fontdict)
#     ax_spectrogram.set_ylabel('Freq (Hz)', fontdict=fontdict)
    

# # plot_sig_spectrogram(sig=ecg_filt, FS=FS_ECG)

# def get_spetral(sig):

#     N_window = sig.shape[0]

#     sig_spectral = np.zeros((N_window//2, sig.shape[1]))
#     T = 1/FS_ACC
#     sig_f = np.linspace(0.0, 1.0/(2.0*T), int(N_window/2))

#     for i in range(sig.shape[1]):
#         v_sig = sig[:,i]
#         yf = scipy.fftpack.fft(v_sig)
#         yf_scaled = 2.0/N * np.abs(yf[:N_window//2])
#         sig_spectral[:,i] = yf_scaled

#     return sig_f, sig_spectral

# ecg_f, ecg_spectral = get_spetral(ecg_beats)

# def plot_spectral(xf, yf, freq_show=100):

#     fig = plt.figure(figsize=(5, 5), dpi=80)
#     fontdict = {'size': 15}

#     ax = fig.add_subplot(1, 1, 1)

#     # freq_show = 100 # Hz
#     yf = yf[xf<freq_show,:]
#     xf = xf[xf<freq_show]

#     ax.plot(xf, yf, color='skyblue', alpha=0.3)
#     ax.plot(xf, yf.mean(axis=1), color='steelblue')

#     ax.scatter(xf[yf.mean(axis=1).argmax()], yf.mean(axis=1).max(), s=35, color='steelblue', zorder=5)
    
#     ax.set_title( 'Frequency domain', fontdict=fontdict)
#     ax.set_xlabel('frequency (Hz)', fontdict=fontdict)
#     ax.set_ylabel('mag (a.u.)', fontdict=fontdict)
    
# # plot_spectral(acc_f, acc_spectral[:,0:50], freq_show=100)
# def get_envelope(sig):
#     analytic_signal = hilbert(sig)
#     amplitude_envelope = np.abs(analytic_signal)
#     return amplitude_envelope